In [ ]:
import numpy as np
import sys
from sklearn import preprocessing

In [ ]:
from misc import load, load_class_labels

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 0.5
train_info = load(dataset_root + 'train_info_dirty.csv', 1, sample_rate)
val_info = load(dataset_root + 'val_info.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
from nn_filter import NN_filter

filter = NN_filter(train_info, train_dir, 'resnet', batch_size=20)

In [ ]:
#filter.check_gpu()

In [ ]:
features, labels, image_names = filter.fit_train()

## Outlier detection using pca on NN features

In [27]:
pca = filter.fit_pca(features, components=513)

In [ ]:
significance_threshold = 0.7
np.set_printoptions(threshold=sys.maxsize)
cum_prod = filter.analyze_pca_components(pca)

cum_prod = [pair for pair in cum_prod if pair[1] > significance_threshold][0]
print('num of component that should be used: ', cum_prod[0])

In [ ]:
new_space = filter.pca_features(pca, features);

In [ ]:
from misc import group_stats
mean, cov = group_stats(new_space, labels)

In [ ]:
from image_discard import find_outliers_per_class
names, distances = find_outliers_per_class(mean, cov, new_space, labels, image_names)

In [ ]:
from image_discard import write_discarded_images
write_discarded_images(names, class_labels, "./discarded_pca_nn_features/", train_dir)

## Outlier detection with NN features

In [ ]:
from image_discard import find_outliers_per_class_centered_scaled

std_features = new_space;
# normalization over the features
std_features = preprocessing.normalize(std_features, axis=0)

name, distances = find_outliers_per_class_centered_scaled(std_features, labels, image_names);

In [ ]:
from image_discard import write_discarded_images
write_discarded_images(names, class_labels, "./discarded_scaling_features/", train_dir)

## Outlier detection with KNN

In [ ]:
use_pca = False

In [ ]:
knn = filter.fit_knn(features, labels, use_pca);

In [ ]:
filter.filter_with_knn(knn, use_pca);

In [ ]:
pred = [x[0] for x in sorted(enumerate(prob), key=lambda x: x[1])[-5:]]
198 in pred